# In this notebook we generate data that will be used by the API

## Data for Genre Function

In [1]:
import pandas as pd
path = 'TransformedData/'
fn_steamGames = path + 'df_steamGames.csv'
df_steamGames = pd.read_csv(fn_steamGames)
df_steamGames

,genres,app_name,release_date,price,id,developer
0,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,2018-01-04,4.99,761140.0,Kotoshiro
1,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,2018-01-04,0.00,643980.0,Secret Level SRL
2,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,2017-07-24,0.00,670290.0,Poolians.com
3,"['Action', 'Adventure', 'Casual']",弹炸人2222,2017-12-07,0.99,767400.0,彼岸领域
4,"['Action', 'Indie', 'Casual', 'Sports']",Log Challenge,2016-04-21,2.99,773570.0,NaN
...,...,...,...,...,...,...
32128,"['Casual', 'Indie', 'Simulation', 'Strategy']",Colony On Mars,2018-01-04,1.99,773640.0,"Nikita ""Ghost_RUS"""
32129,"['Casual', 'Indie', 'Strategy']",LOGistICAL: South Africa,2018-01-04,4.99,733530.0,Sacada
32130,"['Indie', 'Racing', 'Simulation']",Russian Roads,2018-01-04,1.99,610660.0,Laush Dmitriy Sergeevich
32131,"['Casual', 'Indie']",EXIT 2 - Directions,2017-09-02,4.99,658870.0,"xropi,stev3ns"


In [2]:
import ast

def Union(lst1, lst2):
    final_list = list(set(lst1) | set(lst2))
    return final_list

Genres = []
genresSeries = df_steamGames['genres'].values
for g in genresSeries:
    try:
        gl = ast.literal_eval(g)
        Genres = Union(Genres,gl)
    except:
        pass

len(Genres)



303

In [3]:
fn_userItems_R = 'TransformedData/df_userItems_R.csv'
df_userItems_R = pd.read_csv(fn_userItems_R)
ptf = df_userItems_R.groupby('item_id').sum()['playtime_forever'].values
i = df_userItems_R.groupby('item_id').sum().index
df_itemsPtf = pd.DataFrame({'item_id':i,'playtime_forever':ptf})
df_itemsPtf

,item_id,playtime_forever
0,10,17107858
1,20,960524
2,30,756375
3,40,154424
4,50,726545
...,...,...
10973,528580,0
10974,528660,182
10975,529670,0
10976,529820,0


In [4]:
PlayTimeForever = []
it = 1
n = len(Genres)
for G in Genres:
    print(it/n*100,G)
    it+=1
    ids = []
    for i in range(len(df_steamGames)):
        try:
            if G in df_steamGames.loc[i,'genres']:
                ids.append(df_steamGames.loc[i,'id'])
        except:
            pass
    PlayTimeForever.append(df_itemsPtf[df_itemsPtf['item_id'].isin(ids)]['playtime_forever'].sum())

0.33003300330033003 Cartoon
0.6600660066006601 Bikes
0.9900990099009901 Great Soundtrack
1.3201320132013201 3D Platformer
1.65016501650165 Gore
1.9801980198019802 Basketball
2.31023102310231 Dark Humor
2.6402640264026402 Experimental
2.9702970297029703 Golf
3.3003300330033 Competitive
3.6303630363036308 Comic Book
3.9603960396039604 Animation & Modeling
4.29042904290429 Action
4.62046204620462 Documentary
4.9504950495049505 Gambling
5.2805280528052805 Simulation
5.6105610561056105 Sci-fi
5.9405940594059405 Sniper
6.270627062706271 Psychological
6.6006600660066 Hack and Slash
6.9306930693069315 Female Protagonist
7.2607260726072615 Beat 'em up
7.590759075907591 Crafting
7.920792079207921 Batman
8.25082508250825 Naval
8.58085808580858 Quick-Time Events
8.91089108910891 Mature
9.24092409240924 Software Training
9.570957095709572 Team-Based
9.900990099009901 Replay Value
10.231023102310232 Robots
10.561056105610561 Video Production
10.891089108910892 Real Time Tactics
11.221122112211221 Be

In [10]:
genresRank = pd.DataFrame({'genre':Genres,'playtime_forever':PlayTimeForever})
genresRank.sort_values('playtime_forever',ascending=False,ignore_index=True,inplace=True)
genresRank

,genre,playtime_forever
0,Action,3229885934
1,Indie,1484043280
2,RPG,1036023668
3,Simulation,1008801511
4,Adventure,1008174548
...,...,...
298,NSFW,0
299,Superhero,0
300,America,0
301,Lemmings,0


In [19]:
genresRank.to_csv('APIData/genresRank.csv',index=False)

## Data for User for Genre Function

In [30]:
it = 1
n = len(Genres)
for G in Genres:
    print(it/n*100,G)
    it+=1
    ids = []
    for i in range(len(df_steamGames)):
        try:
            if G in df_steamGames.loc[i,'genres']:
                ids.append(df_steamGames.loc[i,'id'])
        except:
            pass
    gbu = df_userItems_R[df_userItems_R['item_id'].isin(ids)].groupby('steam_id').sum()
    gptf = gbu['playtime_forever'].values
    ind = gbu.index
    df_gptf = pd.DataFrame({'steam_id':ind,'playtime_forever':gptf})
    df_gptf.sort_values('playtime_forever',ascending=False,ignore_index=True,inplace=True)

    fn_genresData_i = 'APIData/GenresData/genreData_' + G + '.csv'
    df_gptf.to_csv(fn_genresData_i,index=0)

0.33003300330033003 Cartoon
0.6600660066006601 Bikes
0.9900990099009901 Great Soundtrack
1.3201320132013201 3D Platformer
1.65016501650165 Gore
1.9801980198019802 Basketball
2.31023102310231 Dark Humor
2.6402640264026402 Experimental
2.9702970297029703 Golf
3.3003300330033 Competitive
3.6303630363036308 Comic Book
3.9603960396039604 Animation & Modeling
4.29042904290429 Action
4.62046204620462 Documentary
4.9504950495049505 Gambling
5.2805280528052805 Simulation
5.6105610561056105 Sci-fi
5.9405940594059405 Sniper
6.270627062706271 Psychological
6.6006600660066 Hack and Slash
6.9306930693069315 Female Protagonist
7.2607260726072615 Beat 'em up
7.590759075907591 Crafting
7.920792079207921 Batman
8.25082508250825 Naval
8.58085808580858 Quick-Time Events
8.91089108910891 Mature
9.24092409240924 Software Training
9.570957095709572 Team-Based
9.900990099009901 Replay Value
10.231023102310232 Robots
10.561056105610561 Video Production
10.891089108910892 Real Time Tactics
11.221122112211221 Be

In [6]:
import pandas as pd
G = 'Dark'
fn_genresData_i = 'APIData/GenresData/genreData_' + G + '.csv'
pd.read_csv(fn_genresData_i)

,steam_id,playtime_forever
0,76561198018684159,1630
1,76561197995214384,1207
2,76561198084332433,1137
3,76561198063197045,1121
4,76561198035362505,1114
...,...,...
1111,76561198028431608,0
1112,76561198028550414,0
1113,76561198028584269,0
1114,76561198028686111,0


## Data for Developer Function

In [9]:
import pandas as pd
fn_steamGames = 'TransformedData/df_steamGames.csv'
df_steamGames = pd.read_csv(fn_steamGames)
df_steamGames.drop(columns=['genres','app_name'],inplace=True)
df_steamGames.to_csv('APIData/df_steamGames.csv',index=False)
df_steamGames


,release_date,price,id,developer
0,2018-01-04,4.99,761140.0,Kotoshiro
1,2018-01-04,0.00,643980.0,Secret Level SRL
2,2017-07-24,0.00,670290.0,Poolians.com
3,2017-12-07,0.99,767400.0,彼岸领域
4,2016-04-21,2.99,773570.0,NaN
...,...,...,...,...
32128,2018-01-04,1.99,773640.0,"Nikita ""Ghost_RUS"""
32129,2018-01-04,4.99,733530.0,Sacada
32130,2018-01-04,1.99,610660.0,Laush Dmitriy Sergeevich
32131,2017-09-02,4.99,658870.0,"xropi,stev3ns"


In [14]:
df_steamGames.groupby('developer').count().sort_values('price')

,release_date,price,id
developer,,,
+7 Software,1,1,1
Novus Res,1,1,1
Novo Mundo,1,1,1
"Noviy Disk,Lesta Studios Inc",1,1,1
Nova-box,1,1,1
...,...,...,...
Paradox Development Studio,156,156,156
"KOEI TECMO GAMES CO., LTD.",232,232,232
Dovetail Games,253,253,253


## Data for Sentiment Analysis Function

In [2]:
import pandas as pd
fn_steamGames = 'APIData/df_steamGames.csv'
df_steamGames = pd.read_csv(fn_steamGames)
df_steamGames['release_date'] = pd.to_datetime(df_steamGames['release_date']).dt.year
ids = df_steamGames[df_steamGames['release_date']==2018]['id'].values
ids
#df_steamGames

array([761140., 643980., 772540., 774276., 774277., 774278., 768800.,
       770380., 768880., 765320., 773650., 774271., 773690., 774279.,
       768060., 653960., 777910., 520680., 690410., 766850., 413120.,
       754360., 716110., 659780., 566490., 505330., 597230., 666140.,
       604330., 596580., 617620., 420560., 684700., 698770., 687630.,
       599140., 695810., 658900., 718590., 644000., 604550., 573320.,
       709280., 728170., 754020., 709010., 672240., 596260., 602600.,
       772430., 772400., 770510., 752570., 690200., 529060., 751720.,
       619150., 622440., 646210., 764510., 758090., 757480., 670930.,
       645530., 756640., 598640., 754270., 590820., 695970., 611710.,
       659150., 738270., 680590., 371140., 720250., 733110., 723130.,
       711760., 680360., 546120., 579180., 630720., 659720., 707950.,
       691810., 420170., 409350., 470090., 682670., 628410., 612570.,
       620210., 615530., 608730., 556680., 466450., 556740., 492840.,
       355400., 4033

In [4]:
fn_reviews = 'APIData/df_reviews.csv'
df_reviews = pd.read_csv(fn_reviews)
SIDs = df_reviews['steam_id'].values
SIDs

array([76561197970982479, 76561198035864385, 76561198007712555, ...,
       76561198310819422, 76561198312638244, 76561198313816521])

In [10]:
import pandas as pd
df_reviews_r = pd.DataFrame()
for sid in SIDs:
    fn_reviews_i = 'APIData/ReviewsData/revData_'+str(sid)+'.csv'
    try:
        df_reviews_i = pd.read_csv(fn_reviews_i)
        df_reviews_i.drop(columns=['recommend','posted'],inplace=True)
        df_reviews_r = pd.concat([df_reviews_r,df_reviews_i],ignore_index=True)
    except:
        pass
df_reviews_r.to_csv('APIData/df_reviews_r.csv')
df_reviews_r

,posted,item_id,sentiment_analysis
0,2011-11-05,1250,2
1,2011-07-15,22200,2
2,2011-04-21,43110,1
3,2014-06-24,251610,2
4,2013-09-08,227300,1
...,...,...,...
57675,2016-07-10,70,2
57676,2016-07-08,362890,2
57677,2016-07-03,273110,1
57678,2016-07-20,730,2
